# Advanced Usage

## Complex Types

We have already seen how to define classes / models using simple types, such as `int`, `float`, `str`, `bool`, etc. However in practice data structures are often more complex. They include for examples dictionaries, list of lists, or mutiple allowed types. Of course Pydantic also supports these more complex types, such as lists, dictionaries, enums, and unions. In the following we will see an overview of those types and how to use them:


### Typed Lists and Dictionaries

Lists and dictionaries are very common data structures in Python. Pydantic supports typed lists and dictionaries, which means that we can also define the type of the elements in the list or the type of the values in the dictionary.
Typed lists and dictionaries are defined using the `list` and `dict` generic types. For example, we can define a model with a list of floats as follows:

In [ ]:
from pydantic import BaseModel

class LineV1(BaseModel):
    """A Line object that can be used to represent a line."""
    x: list[float]
    y: list[float]

    def length(self):
        """Length of the line"""
        length = 0

        for idx in range(len(self.x) - 1):
            length += ((self.x[idx] - self.x[idx + 1]) ** 2 + (self.y[idx] - self.y[idx + 1]) ** 2) ** 0.5
        
        return length

In [ ]:
line_v1 = LineV1(x=[0, 1, 3], y=[0, 1, 2])
display(line_v1)

In [ ]:
print(line_v1.length())

Note that the behavior is exactly the same as for simple types. So values are converted to the specified type if possible, and otherwise a `ValidationError` is raised.

In [ ]:
line_v1 = LineV1(x=[0, 1, "3"], y=[0, True, 2])
display(line_v1)

### Enums and Union Types

In manny cases it is useful to provide users with a selection of valid values, such as strings. The data structure to handle this is called `Enum`. Enums are defined using the `Enum` generic type in Python. For example, we can define a selection for the line color:


In [ ]:
from pydantic import BaseModel
from enum import Enum

class LineColor(str, Enum):
    """Line color enum"""
    red = "red"
    green = "green"
    blue = "blue"


class LineV2(BaseModel):
    """A Line object that can be used to represent a line."""
    x: list[float]
    y: list[float]
    color: LineColor = LineColor.red

    def length(self):
        """Length of the line"""
        length = 0

        for idx in range(len(self.x) - 1):
            length += ((self.x[idx] - self.x[idx + 1]) ** 2 + (self.y[idx] - self.y[idx + 1]) ** 2) ** 0.5
        
        return length

In [ ]:
line_v2 = LineV2(x=[0, 1, 2], y=[0, 1, 2], color="red")
display(line_v2) 

Now let's try an invalid color:

In [ ]:
line_v2 = LineV2(x=[0, 1, 2], y=[0, 1, 2], color="purple")

Note that there is https://github.com/pydantic/pydantic-extra-types, which provides support for validation of CSS colors. `from pydantic.color import Color`

In [ ]:
from typing import Optional, Union

from pydantic import BaseModel
from enum import Enum

class LineColor(str, Enum):
    """Line color enum"""
    red = "red"
    green = "green"
    blue = "blue"


class LineV2(BaseModel):
    """A Line object that can be used to represent a line."""
    x: list[float]
    y: list[float]
    color: Union[LineColor, None] = LineColor.red



### Datetime Types

Pydantic supports parsing and validation of datetime types.



In [ ]:
from datetime import datetime

class LonLatTimeVector(BaseModel):
    """Represent a position on earth with time."""
    lon: float
    lat: float
    time: datetime = None

### Custom Data Types

### Hierarchical Structures

One of the most powerful features of Pydantic is the ability to combine models to create hierarchical structures. This is done by defining a model attribute as another model.


In [ ]:
from pydantic import BaseModel

class Point(BaseModel):
    """Representation of a two-dimensional point coordinate."""

    x: float
    y: float

    def distance_to(self, other: "Point") -> float:
        """Computes the distance to another `PointV3`."""
        dx = self.x - other.x
        dy = self.y - other.y
        return (dx**2 + dy**2)**0.5


class Triangle(BaseModel):
    """Representation of a triangle"""
    point_a : Point
    point_b : Point
    point_c : Point

    def circumference(self):
        """Circumference of the triangle"""
        return (
            self.point_a.distance_to(self.point_b)
            + self.point_b.distance_to(self.point_c)
            + self.point_c.distance_to(self.point_a)
        )

triangle = Triangle(
    point_a=Point(x=0, y=0),
    point_b=Point(x=1, y=0),
    point_c=Point(x=0, y=1),
)

display(triangle)

In [ ]:
print(triangle.circumference())

In [ ]:
class LineV3(BaseModel):
    """Line object"""
    points = list[Point]

    def length(self):
        """Line length"""
        length = 0

        for point, next_point in zip(self.points[:-1], self.points[1:]):
            length += point.distance_to(next_point)
        
        return length


If you compare to our first implemmentation at the beginning the `LineV3` is more compact, better readable and more elegant. 

### Private Attributes

By default all attributes of a Pydantic model are public. However, sometimes we want to define private attributes, which are not accessible from outside the model. This can be done using the `ClassVar` and `PrivateAttr` types. The `ClassVar` type is used to define class variables, which are shared by all instances of the class. The `PrivateAttr` type is used to define private attributes, which are not accessible from outside the model. The following example shows how to use these types:

In [ ]:
class :
    pass

`Config.underscore_attrs_are_private` is `True`

## Type Validation

## Dynamic Model Definition

In rare cases it can be useful to create a Pydantic model dynamically, i.e. without defining the model as a class explicitely. This is useful for situatioms where the shape of a model is not known until runtime.. Pydantic supports this use case using the `create_model` function. The function takes the name of the model as the first argument, followed by the fields of the model. The fields are defined as a dictionary, where the keys are the names of the fields and the values are the types of the fields. The types can be either Python types or Pydantic models. The function returns a Pydantic model class.

In [ ]:


list(combinations([1, 2, 3], 2))

Creating models from `TypedDict` and `NamedTuple` is also supported. The following example shows how to create a model from a `TypedDict`: